# Overview

Nothing to say , just run run run  it 

# Step 1 Upload your *workspace* to Google Drive


1. Create a directory called ***DeepFaceLab*** in the root directory of Google Cloud.
2. Upload ***workspace*** to ***DeepFaceLab***  
3. Mount Google Drive as folder
4. Enter the directory by command



In [0]:
#Mount Google Drive as folder

from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Enter the directory 
%cd /content/drive/My Drive/DeepFaceLab/

/content/drive/My Drive/DeepFaceLab


#Step 2  Install DFL


In [0]:
#Clone stable master branch(recommended)

!git clone https://github.com/lbfs/DeepFaceLab_Linux.git

In [0]:
# Into DeepFaceLab_Colab
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Linux

#Pull updates from repository
!git pull


In [0]:
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab

#Install requirements
#find bug tenser version must 1.13.1
!pip install -r requirements.txt
!pip install --upgrade scikit-image

# Step 3. Extract faces

In [0]:
# Into DeepFaceLab_Colab

%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Linux

/content/drive/My Drive/DeepFaceLab/DeepFaceLab_Linux


In [0]:
#Extract frames from DST video

!python main.py videoed extract-video --input-file ../workspace/data_dst.mp4 --output-dir ../workspace/data_dst/

In [0]:
#Extract Face from data_dst

!python main.py extract --input-dir ../workspace/data_dst --output-dir ../workspace/data_dst/aligned --detector s3fd --debug-dir ../workspace/data_dst/aligned_debug

In [0]:
#Sort by histogram DST faces

!python main.py sort --input-dir ../workspace/data_dst/aligned --by hist

In [0]:
#Extract frames from SRC video

!python main.py videoed extract-video --input-file ../workspace/data_src.mp4 --output-dir ../workspace/data_src/

In [0]:
#Extract Face from data_src

!python main.py extract --input-dir ../workspace/data_src --output-dir ../workspace/data_src/aligned --detector s3fd --debug-dir ../workspace/data_src/aligned_debug

In [0]:
#Sort by histogram SRC faces

!python main.py sort --input-dir ../workspace/data_src/aligned --by hist

# Step 4. Train model


*  Not only SAE, H128 is also possible
*   For stop training you need manually interrupt execution. You get error in output, but model will be saved.


In [0]:
# Running trainer.

!python main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model H128 --no-preview

Running trainer.

Loading model...
Press enter in 2 seconds to override model settings./usr/lib/python3.6/multiprocessing/semaphore_tracker.py:143: UserWarning: semaphore_tracker: There appear to be 1 leaked semaphores to clean up at shutdown
  len(cache))
Using TensorFlow backend.
Instructions for updating:
Colocations handled automatically by placer.
Loading: 100% 6004/6004 [28:40<00:00,  3.65it/s]
Loading: 100% 396/396 [01:49<00:00,  3.48it/s]
===== Model summary =====
== Model name: H128
==
== Current iteration: 54208
==
== Model options:
== |== write_preview_history : True
== |== batch_size : 16
== |== sort_by_yaw : False
== |== random_flip : True
== |== lighter_ae : False
== |== pixel_loss : True
== Running on:
== |== [0 : Tesla K80]
Starting. Press "Enter" to stop training and save model.
Instructions for updating:
Use tf.cast instead.
[07:52:56][#054663][1784ms] loss_src:0.060 loss_dst:0.032
[08:07:58][#055140][1781ms] loss_src:0.085 loss_dst:0.035
[08:22:59][#055616][1769ms] l

In [0]:
# Stop tain ，Run this block ,Show last preview image

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os
from google.colab import files
 
print(tf.__version__)

imgpath="/content/drive/My Drive/DeepFaceLab/workspace/model/SAE_history/"
tlist=os.listdir(imgpath)
tlist.sort(key=lambda x:int(x[:-5]))
lastpic=tlist[-1]

image_raw = tf.gfile.GFile(imgpath+lastpic,'rb').read()   #bytes
img = tf.image.decode_jpeg(image_raw)  #Tensor

plt.rcParams['figure.figsize'] = (8, 4.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray' 
plt.rcParams['savefig.dpi'] =200 
plt.rcParams['figure.dpi'] = 200



with tf.Session() as sess:
    plt.imshow(img.eval())
 

# Step 5. Convert frames

In [0]:
#Convert 
!python main.py convert --input-dir ../workspace/data_dst --output-dir ../workspace/data_dst/merged --aligned-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model H128

In [0]:
#Convert frames to result video 

!python main.py videoed video-from-sequence --input-dir ../workspace/data_dst/merged --output-file ../workspace/result.mp4 --reference-file ../workspace/data_dst.mp4

# Others

You can see the results of each step on google Drive

https://drive.google.com/drive/my-drive

In [0]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.0.130-1_amd64.deb

In [0]:
!sudo dpkg -i cuda-repo-ubuntu1804_10.0.130-1_amd64.deb

In [0]:
!sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub

In [0]:
!sudo apt-get update

In [0]:
!sudo apt-get install --no-install-recommends nvidia-driver-418